In [2]:
import numpy as np
import pickle
import time
import sys
import cv2
import math
from scipy.signal import medfilt
from scipy.ndimage.filters import maximum_filter as maxfilt

def PointCloud2Image(M,Sets3DRGB,viewport,filter_size):

    # setting yp output image
    print("...Initializing 2D image...")
    top = viewport[0]
    left = viewport[1]
    h = viewport[2]
    w = viewport[3]
    bot = top  + h + 1
    right = left + w +1;
    output_image = np.zeros((h+1,w+1,3));    

    for counter in range(len(Sets3DRGB)):
        print("...Projecting point cloud into image plane...")

        # clear drawing area of current layer
        canvas = np.zeros((bot,right,3))

        # segregate 3D points from color
        dataset = Sets3DRGB[counter]
        P3D = dataset[:3,:]
        color = (dataset[3:6,:]).T
        
        # form homogeneous 3D points (4xN)
        len_P = len(P3D[1])
        ones = np.ones((1,len_P))
        X = np.concatenate((P3D, ones))
        
        
        # apply (3x4) projection matrix
  
        x = np.matmul(M,X)
        
        # normalize by 3rd homogeneous coordinate
        x = np.around(np.divide(x, np.array([x[2,:],x[2,:],x[2,:]])))

        # truncate image coordinates
        x[:2,:] = np.floor(x[:2,:])

        # determine indices to image points within crop area
        i1 = x[1,:] > top
        i2 = x[0,:] > left
        i3 = x[1,:] < bot
        i4 = x[0,:] < right
        ix = np.logical_and(i1, np.logical_and(i2, np.logical_and(i3, i4)))

        # make reduced copies of image points and cooresponding color
        rx = x[:,ix]
        rcolor = color[ix,:]

        for i in range(len(rx[0])):
            canvas[int(rx[1,i]),int(rx[0,i]),:] = rcolor[i,:]

        # crop canvas to desired output size
        cropped_canvas = canvas[top:top+h+1,left:left+w+1]

        # filter individual color channels
        shape = cropped_canvas.shape
        filtered_cropped_canvas = np.zeros(shape)
        print("...Running 2D filters...")
        for i in range(3):
            # max filter
            filtered_cropped_canvas[:,:,i] = maxfilt(cropped_canvas[:,:,i],5)

        
        # get indices of pixel drawn in the current canvas
        drawn_pixels = np.sum(filtered_cropped_canvas,2)
        idx = drawn_pixels != 0
        shape = idx.shape
        shape = (shape[0],shape[1],3)
        idxx = np.zeros(shape,dtype=bool)

        # make a 3-channel copy of the indices
        idxx[:,:,0] = idx
        idxx[:,:,1] = idx
        idxx[:,:,2] = idx

        # erase canvas drawn pixels from the output image
        output_image[idxx] = 0

        #sum current canvas on top of output image
        output_image = output_image + filtered_cropped_canvas

    print("Done")
    return output_image


def get_rotation_and_translation(angle):
    
    start_x = -0.18 #Bringing camera bit to the left to center the foreground object
    start_z = 4 #Start position of the camer
    final_radius = 2.5 #Final radius of the half circle
    
    
    
    rot_angle = angle
    
    # 3x3 Y-axis rotation matrix
    R = np.array([[math.cos(math.radians(rot_angle)), 0, math.sin(math.radians(rot_angle))],
                  [0, 1, 0],
                  [-math.sin(math.radians(rot_angle)), 0 ,math.cos(math.radians(rot_angle))]])
    
    # Calculation of the final position of camera before axis rotation is performed
    theta = 90
    final_x = final_radius*math.cos(math.radians(theta + angle))
    final_z = final_radius*math.sin(math.radians(theta + angle))
    
    # Performing axis rotation and calculating the co-ordinates of the final position based on the rotated axis
    cam_final_x = ((final_x) * math.cos(math.radians(angle))) + ((final_z) * math.sin(math.radians(angle)))
    cam_final_z = ((-final_x) * math.sin(math.radians(angle))) + ((final_z) * math.cos(math.radians(angle)))
    
    # Performing axis rotation and calculating the co-ordinates of the Start position based on the rotated axis
    cam_x = ((start_x) * math.cos(math.radians(angle))) + ((start_z) * math.sin(math.radians(angle)))
    cam_z = ((-start_x) * math.sin(math.radians(angle))) + ((start_z) * math.cos(math.radians(angle)))
 
    #Returning the Rotation matrix and the X, Z translation that the camera needs to perform based on the new axis plane
    return R, round(cam_final_x - cam_x,3), round(cam_final_z - cam_z,3)

def generate_video(image_dir):
    import os
    files = os.listdir(image_dir)
    files.sort(key=lambda f: int(f.split('.')[0]))
    
    img = cv2.imread(image_dir + files[0])
    video = cv2.VideoWriter("Half_Circle_Video.wmv", cv2.VideoWriter_fourcc(*'DIVX'), 5, (img.shape[1], img.shape[0]))
    video.write(img)
    
    for file in files[1:]:
        file = image_dir + file
        img = cv2.imread(file)
        video.write(img)
    video.release()
    cv2.destroyAllWindows()

def main():
    file_p = open("Input_data/data.obj",'rb')
    camera_objs = pickle.load(file_p)

    # extract objects from object array
    crop_region = camera_objs[0].flatten()
    filter_size = camera_objs[1].flatten()
    K = camera_objs[2]
    ForegroundPointCloudRGB = camera_objs[3]
    BackgroundPointCloudRGB = camera_objs[4]
    
    t = np.array([0, 0.25, 0]).reshape((3,1))
    # X, Z co-ordinates of Translation matrix would be determined later based on camera angle 

    # create variables for computation
    data3DC = (BackgroundPointCloudRGB,ForegroundPointCloudRGB)
    count = 0
    all_images = []
    
    #FOR LOOP iterating over each angle of camera.
    for angle in range(-90, 95, 5):
        count += 1
        image_dir = ".\\Output_Images\\"
        fname = image_dir + "\{}.jpg".format(angle)
        print("\nGenerating {}".format(fname))
        
        # The following function generates the rotation and translation matrix
        R, t[0], t[2] = get_rotation_and_translation(angle)
        
        
        M = np.matmul(K,(np.hstack((R,t))))
        img = PointCloud2Image(M,data3DC,crop_region,filter_size)
        
        # Convert image values form (0-1) to (0-255) and cahnge type from float64 to float32
        img = 255*(np.array(img, dtype=np.float32))
        
        all_images.append(img)
        # convert image from RGB to BGR for OpenCV
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        box_start = (1536-200, 1024-320)
        box_end = (1536+200, 1024+320)
        img_bgr = cv2.rectangle(img_bgr, box_start, box_end, (0, 255, 0), 5)
        img_bgr = cv2.circle(img_bgr, (1536,1024), 5, (0, 255, 0), 5)
        
        
        
        cv2.imwrite(fname, img_bgr)
    
    print("Generating Video")
    generate_video(image_dir)
    print("---- Done ----")
    
    return all_images


In [3]:
main()


Generating .\Output_Images\\-90.jpg
...Initializing 2D image...
...Projecting point cloud into image plane...
...Running 2D filters...
...Projecting point cloud into image plane...
...Running 2D filters...
Done

Generating .\Output_Images\\-85.jpg
...Initializing 2D image...
...Projecting point cloud into image plane...
...Running 2D filters...
...Projecting point cloud into image plane...
...Running 2D filters...
Done

Generating .\Output_Images\\-80.jpg
...Initializing 2D image...
...Projecting point cloud into image plane...
...Running 2D filters...
...Projecting point cloud into image plane...
...Running 2D filters...
Done

Generating .\Output_Images\\-75.jpg
...Initializing 2D image...
...Projecting point cloud into image plane...
...Running 2D filters...
...Projecting point cloud into image plane...
...Running 2D filters...
Done

Generating .\Output_Images\\-70.jpg
...Initializing 2D image...
...Projecting point cloud into image plane...
...Running 2D filters...
...Projecting po

[array([[[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        ...,
 
        [[ 85.,  93., 119.],
         [ 86.,  93., 119.],
         [ 87.,  93., 119.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[ 85.,  93., 119.],
         [ 86.,  93., 119.],
         [ 86.,  93., 119.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],
 
        [[ 85.,  93., 119.],
 